In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import gensim
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
from transformers import *
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import os
import time
os.environ["TF_KERAS"]='1'

#### Import des données nettoyées

In [2]:
#Import des données
df = pd.read_csv('cleaned.csv')
df.head()

,Id,body_title_bow
0,39946022,how make horizontal scrolling collectionview f...
1,57086301,tried write custom implementation basic neural...
2,21965,could somebody please rundown programmatically...
3,22067,has anybody used project gantt chart control y...
4,38281760,found example how refresh oauth token moya rxs...


In [ ]:
#import des tags
vectorized_tags = pd.read_csv('vectorized_tags.csv')

## Modélisation

In [4]:
#Définission de nos modèles BoW
feat = 'body_title_bow'
#Countvectorizer
tfvect = CountVectorizer(stop_words='english', max_df=0.95, min_df=1)
tf_body = tfvect.fit_transform(df[feat])

#TF-IDF
tfidfvect = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=1)
tfidf_body = tfidfvect.fit_transform(df[feat])


### Count vectorizer

In [ ]:
#Split des données
X = tf_body
y = vectorized_tags.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

#### Fit sur nos données

In [8]:
#fit sur les données
lr = LogisticRegression(max_iter=50)
ovr_cv = OneVsRestClassifier(lr)
ovr_cv.fit(X_train, y_train) 

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

OneVsRestClassifier(estimator=LogisticRegression(max_iter=50))

#### Score du classifier

In [9]:
#prediction et score pour notre test
y_pred = ovr_cv.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.67      0.76       144
           1       0.93      0.75      0.83       675
           2       0.72      0.62      0.67       139
           3       0.85      0.57      0.68       158
           4       0.58      0.48      0.52       208
           5       0.82      0.63      0.71       469
           6       0.88      0.69      0.78        88
           7       0.84      0.52      0.64       209
           8       0.71      0.60      0.65       260
           9       0.33      0.21      0.26       219
          10       0.43      0.25      0.32       219
          11       0.93      0.87      0.90       145
          12       0.66      0.40      0.50       158
          13       0.76      0.56      0.65       266
          14       0.56      0.47      0.51       412
          15       0.62      0.38      0.47       162
          16       0.73      0.62      0.67       639
          17       0.56    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
#prediction et score pour notre val
y_val_pred = ovr_cv.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.91      0.67      0.77       121
           1       0.91      0.74      0.82       656
           2       0.79      0.72      0.75       132
           3       0.84      0.50      0.62       137
           4       0.54      0.41      0.46       185
           5       0.84      0.67      0.74       499
           6       0.89      0.75      0.81        64
           7       0.77      0.53      0.63       213
           8       0.81      0.62      0.70       284
           9       0.44      0.27      0.33       230
          10       0.41      0.29      0.34       213
          11       0.93      0.78      0.85       149
          12       0.66      0.44      0.53       171
          13       0.73      0.59      0.65       298
          14       0.59      0.45      0.51       457
          15       0.57      0.43      0.49       134
          16       0.74      0.58      0.65       621
          17       0.62    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Export du countvectorizer pour réutilisation dans notre app
filename = 'API/scripts/cv_model.pkl'
pickle.dump(tfvect, open(filename, 'wb'))
filename = 'API/scripts/ovr_model.pkl'
pickle.dump(ovr_cv, open(filename, 'wb'))

### TFIDF

In [12]:
#Split des données
X = tfidf_body
y = vectorized_tags.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [13]:
#Fit sur nos données
lr = LogisticRegression(max_iter=50)
ovr_tfidf = OneVsRestClassifier(lr)
ovr_tfidf.fit(X_train, y_train) 

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

OneVsRestClassifier(estimator=LogisticRegression(max_iter=50))

#### Score du classifier

In [14]:
#prediction et score pour notre test
y_pred = ovr_tfidf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.31      0.47       144
           1       0.98      0.60      0.75       675
           2       0.80      0.40      0.53       139
           3       0.97      0.19      0.32       158
           4       0.76      0.26      0.39       208
           5       0.92      0.43      0.58       469
           6       1.00      0.39      0.56        88
           7       0.97      0.27      0.42       209
           8       0.87      0.38      0.53       260
           9       0.72      0.08      0.15       219
          10       0.59      0.12      0.20       219
          11       1.00      0.63      0.77       145
          12       0.91      0.20      0.32       158
          13       0.92      0.37      0.53       266
          14       0.70      0.31      0.43       412
          15       0.64      0.18      0.28       162
          16       0.82      0.44      0.57       639
          17       0.73    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
#prediction et score pour notre val
y_val_pred = ovr_tfidf.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.91      0.26      0.40       121
           1       0.97      0.60      0.74       656
           2       0.86      0.42      0.57       132
           3       0.91      0.23      0.37       137
           4       0.63      0.21      0.31       185
           5       0.93      0.47      0.62       499
           6       1.00      0.38      0.55        64
           7       0.91      0.32      0.48       213
           8       0.94      0.43      0.59       284
           9       0.88      0.10      0.17       230
          10       0.53      0.14      0.22       213
          11       0.98      0.66      0.79       149
          12       0.84      0.19      0.31       171
          13       0.87      0.37      0.52       298
          14       0.73      0.31      0.44       457
          15       0.62      0.17      0.27       134
          16       0.82      0.40      0.54       621
          17       0.83    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### LDA

In [16]:
#Optimisation du nombre de topics pour countvectorizer 

topics = [25,26,27,28,29,30,31,32,33,34,35,36,37]
df_perplexity_score = pd.DataFrame(columns=['Topics', 'Perplexity'])

for i in topics :
    lda_tf = LatentDirichletAllocation(n_components=i, random_state=0)
    lda_tf.fit(tf_body)
    perplexity = lda_tf.perplexity(tf_body)
    df_perplexity_score = df_perplexity_score.append({'Topics':str(i), 'Perplexity': perplexity}, ignore_index=True)


C:\Users\Rémi\AppData\Local\Temp\ipykernel_16164\1054146431.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_perplexity_score = df_perplexity_score.append({'Topics':str(i), 'Perplexity': perplexity}, ignore_index=True)
C:\Users\Rémi\AppData\Local\Temp\ipykernel_16164\1054146431.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_perplexity_score = df_perplexity_score.append({'Topics':str(i), 'Perplexity': perplexity}, ignore_index=True)
C:\Users\Rémi\AppData\Local\Temp\ipykernel_16164\1054146431.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_perplexity_score = df_perplexity_score.append({'Topics':str(i), 'Perplexity': perplexity}, ignore_index=True)
C:\Users\Rémi\AppData\Local\Temp\ipykernel_16164\105414

In [ ]:
#Affichage de la courbe de score de perplexité
sns.lineplot(x="Topics", y="Perplexity", data=df_perplexity_score)

In [ ]:
#Optimisation du nombre de topics our tf-idf
topics = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
df_perplexity_score = pd.DataFrame(columns=['Topics', 'Perplexity'])

for i in topics :
    lda_tfidf = LatentDirichletAllocation(n_components=i, random_state=0)
    lda_tfidf.fit(tf_body)
    perplexity = lda_tf.perplexity(tf_body)
    df_perplexity_score = df_perplexity_score.append({'Topics':str(i), 'Perplexity': perplexity}, ignore_index=True)

In [ ]:
#Affichage de la courbe de score de perplexité
sns.lineplot(x="Topics", y="Perplexity", data=df_perplexity_score)

In [19]:
#Split des données
X = tf_body
y = vectorized_tags.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Pour countvectorizer
lda_tf = LatentDirichletAllocation(n_components=33, random_state=0)
lda_tf.fit(X_train)
# Pour TFIDF 
lda_tfidf = LatentDirichletAllocation(n_components=33, random_state=0)
lda_tfidf.fit(tfidf_body)

LatentDirichletAllocation(n_components=33, random_state=0)

#### Visualisation des topics

In [21]:
#Projections en 2D de nos topics
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, X_train, tfvect)

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7      0.119781 -0.049066       1        1  6.268655
10     0.145043  0.008492       2        1  5.687255
32     0.012054  0.189339       3        1  5.527344
15    -0.043414  0.003563       4        1  5.387817
30     0.023081 -0.061281       5        1  5.351976
6     -0.182006  0.070514       6        1  4.826703
14     0.057483  0.242098       7        1  4.726656
12    -0.090433  0.064239       8        1  4.603285
31    -0.147419  0.001890       9        1  4.503540
8      0.027551  0.199344      10        1  4.401797
22     0.036748 -0.074507      11        1  4.011833
1     -0.152136 -0.011177      12        1  3.816664
4     -0.149548 -0.011499      13        1  3.702637
24     0.084228 -0.013283      14        1  3.662545
3     -0.165267 -0.034150      15        1  2.983027
27     0.141349 -0.079813      16        1  2.689789
5     -0.181725 -0.027162      17        1  2.454310
19    -0.066435 -0.014008      18        1  2.395409
26     0.036466  0.048476      19        1  2.321031
2     -0.003996 -0.075935      20        1  2.320979
20    -0.012984 -0.072015      21        1  2.296197
18     0.063760 -0.078629      22        1  2.107696
29    -0.020619 -0.058641      23        1  1.710732
13     0.083896  0.023304      24        1  1.671677
9     -0.022905 -0.020406      25        1  1.495221
28    -0.008456 -0.038194      26        1  1.442749
11     0.127476  0.091757      27        1  1.352883
17     0.101269 -0.088558      28        1  1.304268
21    -0.000877 -0.008525      29        1  1.199973
23     0.122568  0.079386      30        1  1.185344
16    -0.080395 -0.040506      31        1  1.065325
0      0.118663 -0.104171      32        1  0.801393
25     0.027196 -0.060877      33        1  0.723290, topic_info=            Term          Freq         Total Category  logprob  loglift
30636       file  12016.000000  12016.000000  Default  30.0000  30.0000
37913      image   6976.000000   6976.000000  Default  29.0000  29.0000
72308       test   4817.000000   4817.000000  Default  28.0000  28.0000
69005        src   5161.000000   5161.000000  Default  27.0000  27.0000
50595        net   6053.000000   6053.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
31615  following     39.368457   7515.428494  Topic33  -5.9185  -0.3226
46008       make     36.609466   5273.324069  Topic33  -5.9912  -0.0410
63822    running     35.262469   3929.317705  Topic33  -6.0287   0.2157
59854   question     35.191068   5195.617629  Topic33  -6.0307  -0.0657
63773        run     33.062885   5948.337226  Topic33  -6.0931  -0.2633

[2466 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0          4  0.021543          00
0         29  0.244159          00
0         32  0.732478          00
2         15  0.904487        0000
2         27  0.069576        0000
...      ...       ...         ...
82284     11  0.024028        yyyy
82284     29  0.889030        yyyy
82424     33  0.890685     zend_db
82651     22  0.908266        zope
82759     17  0.958936  zygoteinit

[13005 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 11, 33, 16, 31, 7, 15, 13, 32, 9, 23, 2, 5, 25, 4, 28, 6, 20, 27, 3, 21, 19, 30, 14, 10, 29, 12, 18, 22, 24, 17, 1, 26])

In [22]:
lda_tf.perplexity(X_train)

3012.22685541691

In [23]:
#Projection en 2D sur notre test
pyLDAvis.sklearn.prepare(lda_tf, X_test, tfvect)

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7      0.119781 -0.049066       1        1  7.624856
10     0.145043  0.008492       2        1  5.678222
15    -0.043414  0.003563       3        1  5.554521
32     0.012054  0.189339       4        1  5.276685
30     0.023081 -0.061281       5        1  5.252986
31    -0.147419  0.001890       6        1  5.139888
6     -0.182006  0.070514       7        1  4.742201
8      0.027551  0.199344       8        1  4.606675
12    -0.090433  0.064239       9        1  4.573689
14     0.057483  0.242098      10        1  4.512549
22     0.036748 -0.074507      11        1  4.119463
1     -0.152136 -0.011177      12        1  3.881721
4     -0.149548 -0.011499      13        1  3.768202
24     0.084228 -0.013283      14        1  3.590299
27     0.141349 -0.079813      15        1  2.893017
3     -0.165267 -0.034150      16        1  2.787742
5     -0.181725 -0.027162      17        1  2.397593
2     -0.003996 -0.075935      18        1  2.373537
19    -0.066435 -0.014008      19        1  2.202899
26     0.036466  0.048476      20        1  2.182999
20    -0.012984 -0.072015      21        1  2.132207
18     0.063760 -0.078629      22        1  2.024085
29    -0.020619 -0.058641      23        1  1.672574
13     0.083896  0.023304      24        1  1.560141
28    -0.008456 -0.038194      25        1  1.317013
17     0.101269 -0.088558      26        1  1.211444
23     0.122568  0.079386      27        1  1.203920
9     -0.022905 -0.020406      28        1  1.176301
11     0.127476  0.091757      29        1  1.159653
16    -0.080395 -0.040506      30        1  1.067181
21    -0.000877 -0.008525      31        1  1.051076
0      0.118663 -0.104171      32        1  0.708250
25     0.027196 -0.060877      33        1  0.556410, topic_info=           Term         Freq        Total Category  logprob  loglift
30636      file  2984.000000  2984.000000  Default  30.0000  30.0000
37913     image  1681.000000  1681.000000  Default  29.0000  29.0000
72308      test  1199.000000  1199.000000  Default  28.0000  28.0000
69005       src  1250.000000  1250.000000  Default  27.0000  27.0000
50595       net  1460.000000  1460.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
46008      make     7.148937  1349.339435  Topic33  -5.9912  -0.0490
63822   running     6.885901  1027.198490  Topic33  -6.0287   0.1863
59854  question     6.871958  1362.968881  Topic33  -6.0307  -0.0986
53820       org     6.410716   253.682146  Topic33  -6.1002   1.5133
63773       run     6.456376  1560.859990  Topic33  -6.0931  -0.2965

[2468 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0          3  0.031958          00
0         31  0.239682          00
0         32  0.735024          00
2         16  0.883955        0000
2         29  0.098217        0000
...      ...       ...         ...
82284      3  0.106309        yyyy
82284     31  0.850471        yyyy
82424     33  0.631412     zend_db
82651     22  0.855436        zope
82759     17  1.031330  zygoteinit

[11002 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 11, 16, 33, 31, 32, 7, 9, 13, 15, 23, 2, 5, 25, 28, 4, 6, 3, 20, 27, 21, 19, 30, 14, 29, 18, 24, 10, 12, 17, 22, 1, 26])

In [24]:
#Projection en 2D d'un LDA sur tfidf pour contrôle
pyLDAvis.sklearn.prepare(lda_tfidf, tfidf_body, tfidfvect)

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.467356 -0.031250       1        1  33.193805
15    -0.440980 -0.033381       2        1  24.393020
14    -0.329774  0.208873       3        1   8.295618
5     -0.340043 -0.124061       4        1   7.758884
30     0.017372 -0.001436       5        1   1.405128
32     0.022486  0.040122       6        1   1.213947
8      0.047662  0.005227       7        1   1.166915
1      0.055919 -0.000190       8        1   1.041208
21     0.056512 -0.004229       9        1   1.008911
24     0.051169  0.006419      10        1   0.992618
22     0.057426 -0.004381      11        1   0.941937
18     0.057378 -0.003800      12        1   0.938542
11     0.055557  0.004596      13        1   0.935784
7      0.057597 -0.001572      14        1   0.929151
31     0.056843  0.000588      15        1   0.927418
3      0.058064 -0.002763      16        1   0.894711
9      0.057762 -0.003859      17        1   0.892148
6      0.058319 -0.003627      18        1   0.867374
17     0.058194 -0.004452      19        1   0.858789
26     0.057418 -0.003837      20        1   0.857166
28     0.057763 -0.005065      21        1   0.846449
25     0.058553 -0.003698      22        1   0.823282
16     0.057231 -0.002638      23        1   0.822441
10     0.058106 -0.002384      24        1   0.821395
29     0.058230 -0.003787      25        1   0.820556
4      0.057833 -0.002541      26        1   0.820056
13     0.057926 -0.003607      27        1   0.813404
0      0.058104 -0.003567      28        1   0.809217
19     0.057500 -0.004424      29        1   0.800800
20     0.058352 -0.002841      30        1   0.793214
2      0.058205 -0.004178      31        1   0.788132
23     0.057795 -0.002373      32        1   0.768615
27     0.056877 -0.001883      33        1   0.759365, topic_info=           Term        Freq       Total Category  logprob  loglift
18387     class  455.000000  455.000000  Default  30.0000  30.0000
22843      data  504.000000  504.000000  Default  29.0000  29.0000
37913     image  395.000000  395.000000  Default  28.0000  28.0000
43407      lang  171.000000  171.000000  Default  27.0000  27.0000
54262  override  165.000000  165.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
15187      blur    1.397848    8.904004  Topic33  -7.0332   3.0289
14253      beam    0.731903    2.014073  Topic33  -7.6802   3.8682
53945      osgi    0.862106    3.143266  Topic33  -7.5165   3.5868
22091    ctypes    0.758875    3.502588  Topic33  -7.6441   3.3510
82467     zeros    0.694172    4.454054  Topic33  -7.7332   3.0216

[1636 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
1184      12  0.370415      10g
2471       2  0.588361      1mb
2471      24  0.294181      1mb
3875      24  0.683418      303
4108      23  0.944808     3339
...      ...       ...      ...
82642      8  0.216203  zooming
82642     13  0.648608  zooming
82647     13  0.505691    zooms
82718     16  0.447347     zurb
82719     21  0.685282     zuul

[2532 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 16, 15, 6, 31, 33, 9, 2, 22, 25, 23, 19, 12, 8, 32, 4, 10, 7, 18, 27, 29, 26, 17, 11, 30, 5, 14, 1, 20, 21, 3, 24, 28])

In [25]:
lda_tfidf.perplexity(tfidf_body)

93547.13248695762

In [26]:
#Affichage des topics et des n_top_words de ces topics
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda_tf, tfvect.get_feature_names_out(), no_top_words)

Topic 0:
product selenium hash linq 00 attributes products webdriver relationship data
Topic 1:
server php connection rails connect running blockquote ve tried ruby
Topic 2:
blockquote function method java static object variable lambda functions warning
Topic 3:
net windows asp web application microsoft dll iis assembly project
Topic 4:
project studio build visual version files file library folder package
Topic 5:
java eclipse project jar gradle maven log com android org
Topic 6:
app android ios xcode device iphone google application issue problem
Topic 7:
data time ve know question text solution need looking example
Topic 8:
view button bar text set screen change custom controller layout
Topic 9:
application blockquote binding wpf event java bind xaml need property
Topic 10:
string list array value function object values type python return
Topic 11:
images image canvas map points maps google draw line need
Topic 12:
js url component request angular node react app api response
Topic 13

### Word2Vec

In [74]:
#split des données
X = df['body_title_bow']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [76]:
#import du word2vec Google
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True,)

#### Embedding et fit des données

In [77]:
#Vectorisation de nos textes
X_train_vectors = []
for text in X_train:
    text_vectors = []
    for word in text.split():
        if word in model:
            text_vectors.append(model[word])
    X_train_vectors.append(np.mean(text_vectors, axis=0))
X_val_vectors = []
for text in X_val:
    text_vectors = []
    for word in text.split():
        if word in model:
            text_vectors.append(model[word])
    X_val_vectors.append(np.mean(text_vectors, axis=0))
X_test_vectors = []
for text in X_test:
    text_vectors = []
    for word in text.split():
        if word in model:
            text_vectors.append(model[word])
    X_test_vectors.append(np.mean(text_vectors, axis=0))

In [39]:
#Fit des données
ovr.fit(X_train_vectors, y_train) 

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

OneVsRestClassifier(estimator=LogisticRegression(max_iter=50))

#### Score du classifier

In [40]:
#prediction et score pour notre test
y_pred = ovr.predict(X_test_vectors)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.01      0.01       144
           1       0.91      0.35      0.51       675
           2       0.95      0.15      0.26       139
           3       1.00      0.01      0.01       158
           4       0.70      0.13      0.23       208
           5       0.88      0.09      0.17       469
           6       1.00      0.10      0.19        88
           7       1.00      0.05      0.09       209
           8       0.78      0.17      0.27       260
           9       0.29      0.01      0.02       219
          10       0.57      0.06      0.11       219
          11       0.00      0.00      0.00       145
          12       0.50      0.03      0.05       158
          13       0.93      0.15      0.26       266
          14       0.66      0.16      0.26       412
          15       0.72      0.20      0.32       162
          16       0.75      0.20      0.31       639
          17       0.69    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
#prediction et score pour notre val
y_val_pred = ovr.predict(X_val_vectors)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.80      0.03      0.06       121
           1       0.89      0.39      0.54       656
           2       0.85      0.13      0.22       132
           3       1.00      0.02      0.04       137
           4       0.64      0.15      0.24       185
           5       0.92      0.11      0.20       499
           6       1.00      0.19      0.32        64
           7       0.93      0.06      0.11       213
           8       0.79      0.21      0.33       284
           9       1.00      0.02      0.04       230
          10       0.52      0.07      0.12       213
          11       0.00      0.00      0.00       149
          12       0.80      0.05      0.09       171
          13       0.80      0.14      0.24       298
          14       0.64      0.21      0.32       457
          15       0.57      0.16      0.25       134
          16       0.69      0.19      0.29       621
          17       0.59    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### BERT

In [44]:
# Fonction de préparation des sentences
def bert_inp_fct(sentences, bert_tokenizer, max_length) :
    input_ids=[]
    token_type_ids = []
    attention_mask=[]
    bert_inp_tot = []

    for sent in sentences:
        bert_inp = bert_tokenizer.encode_plus(sent,
                                              add_special_tokens = True,
                                              max_length = max_length,
                                              padding='max_length',
                                              return_attention_mask = True, 
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_tensors="tf")
    
        input_ids.append(bert_inp['input_ids'][0])
        token_type_ids.append(bert_inp['token_type_ids'][0])
        attention_mask.append(bert_inp['attention_mask'][0])
        bert_inp_tot.append((bert_inp['input_ids'][0], 
                             bert_inp['token_type_ids'][0], 
                             bert_inp['attention_mask'][0]))

    input_ids = np.asarray(input_ids)
    token_type_ids = np.asarray(token_type_ids)
    attention_mask = np.array(attention_mask)
    
    return input_ids, token_type_ids, attention_mask, bert_inp_tot
    

# Fonction de création des features
def feature_BERT_fct(model, model_type, sentences, max_length, b_size, mode='TFhub') :
    batch_size = b_size
    batch_size_pred = b_size
    bert_tokenizer = AutoTokenizer.from_pretrained(model_type)
    time1 = time.time()

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        input_ids, token_type_ids, attention_mask, bert_inp_tot = bert_inp_fct(sentences[idx:idx+batch_size], 
                                                                  bert_tokenizer, max_length)
    
        if mode=='HF' :    # Bert HuggingFace
            outputs = model.predict([input_ids, attention_mask, token_type_ids], batch_size=batch_size_pred)
            last_hidden_states = outputs.last_hidden_state

        if mode=='TFhub' : # Bert Tensorflow Hub
            text_preprocessed = {"input_word_ids" : input_ids, 
                             "input_mask" : attention_mask, 
                             "input_type_ids" : token_type_ids}
            outputs = model(text_preprocessed)
            last_hidden_states = outputs['sequence_output']
         
        if step ==0 :
            last_hidden_states_tot = last_hidden_states
            last_hidden_states_tot_0 = last_hidden_states
        else :
            last_hidden_states_tot = np.concatenate((last_hidden_states_tot,last_hidden_states))

    if (len(sentences) % batch_size) != 0:
        idx = len(sentences) - (len(sentences) % batch_size)
        input_ids, token_type_ids, attention_mask, bert_inp_tot = bert_inp_fct(sentences[idx:], bert_tokenizer, max_length)

        if mode=='HF' :    # Bert HuggingFace
            outputs = model.predict([input_ids, attention_mask, token_type_ids], batch_size=batch_size_pred)
            last_hidden_states = outputs.last_hidden_state

        if mode=='TFhub' : # Bert Tensorflow Hub
            text_preprocessed = {"input_word_ids" : input_ids, 
                                "input_mask" : attention_mask, 
                                "input_type_ids" : token_type_ids}
            outputs = model(text_preprocessed)
            last_hidden_states = outputs['sequence_output']
        
        last_hidden_states_tot = np.concatenate((last_hidden_states_tot,last_hidden_states))

    features_bert = np.array(last_hidden_states_tot).mean(axis=1)

    time2 = np.round(time.time() - time1,0)
    print("temps traitement : ", time2)
    
    return features_bert, last_hidden_states_tot

In [45]:
#Import du modèle Bert pré entrainé
import tensorflow_hub as hub

# Guide sur le Tensorflow hub : https://www.tensorflow.org/text/tutorials/classify_text_with_bert
model_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
bert_layer = hub.KerasLayer(model_url, trainable=True)

#### Embedding et fit des données

In [46]:
#Paramètrage de BERT
max_length = 64
batch_size = 15
model_type = 'bert-base-uncased'
model = bert_layer
sentences = df['body_title_bow'].to_list()

#Embedding de nos textes
features_bert, last_hidden_states_tot = feature_BERT_fct(model, model_type, sentences, 
                                                         max_length, batch_size, mode='TFhub')

loading configuration file config.json from cache at C:\Users\Rémi/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at C:\Users\Rémi/.cache\huggingface\hub\models--bert-base-uncased\snapshots\0a6aa9128b6194f4f3c4db429b6cb4891cdb421b\vocab.txt

temps traitement :  4870.0


In [69]:
#Normalisation et split des données
X = features_bert

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [70]:
#Fit de notre modèle de classification
lr = LogisticRegression(max_iter=50)
ovr_bert = OneVsRestClassifier(lr)
ovr_bert.fit(X_train, y_train) 

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

OneVsRestClassifier(estimator=LogisticRegression(max_iter=50))

#### Score du classifier

In [71]:
#prediction et score pour notre test
y_pred = ovr_bert.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.48      0.56       144
           1       0.81      0.55      0.65       675
           2       0.55      0.42      0.48       139
           3       0.50      0.24      0.32       158
           4       0.51      0.31      0.39       208
           5       0.75      0.41      0.53       469
           6       0.65      0.58      0.61        88
           7       0.59      0.29      0.39       209
           8       0.57      0.44      0.50       260
           9       0.31      0.10      0.15       219
          10       0.35      0.16      0.22       219
          11       0.84      0.63      0.72       145
          12       0.50      0.22      0.30       158
          13       0.73      0.35      0.47       266
          14       0.57      0.35      0.44       412
          15       0.53      0.39      0.45       162
          16       0.64      0.43      0.52       639
          17       0.51    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [72]:
#prediction et score pour notre val
y_pred_val = ovr_bert.predict(X_val)
print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       0.65      0.50      0.57       121
           1       0.80      0.56      0.66       656
           2       0.49      0.39      0.43       132
           3       0.57      0.26      0.36       137
           4       0.45      0.24      0.32       185
           5       0.74      0.43      0.54       499
           6       0.53      0.52      0.52        64
           7       0.66      0.31      0.43       213
           8       0.59      0.44      0.50       284
           9       0.42      0.12      0.18       230
          10       0.39      0.22      0.28       213
          11       0.82      0.66      0.74       149
          12       0.48      0.19      0.27       171
          13       0.65      0.37      0.47       298
          14       0.59      0.34      0.43       457
          15       0.39      0.30      0.34       134
          16       0.68      0.42      0.52       621
          17       0.51    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### USE

In [83]:
#Import du modèle pré entrainé
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [84]:
def feature_USE_fct(sentences, b_size) :
    batch_size = b_size
    time1 = time.time()

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        feat = embed(sentences[idx:idx+batch_size])

        if step ==0 :
            features = feat
        else :
            features = np.concatenate((features,feat))


    if len(sentences) % batch_size != 0:
        idx = (len(sentences)//batch_size)*batch_size
        remaining_sentences = sentences[idx:]
        remaining_embeddings = embed(remaining_sentences)
        features = np.concatenate((features, remaining_embeddings))

        
    time2 = np.round(time.time() - time1,0)
    return features

#### Embedding et fit des données

In [85]:
#Paramètrage de USE
batch_size = 10
sentences = df['body_title_bow'].to_list()

In [86]:
#Embedding des textes
features_USE = feature_USE_fct(sentences, batch_size)

In [87]:
#Split et normalisation des données

X = features_USE
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [88]:
#Fit de notre classifier
lr = LogisticRegression(max_iter=100)
ovr_use = OneVsRestClassifier(lr)
ovr_use.fit(X_train, y_train) 

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

OneVsRestClassifier(estimator=LogisticRegression())

#### Score du classifier

In [89]:
#prediction et score pour notre test
y_pred = ovr_use.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.44      0.49       144
           1       0.78      0.55      0.64       675
           2       0.46      0.39      0.42       139
           3       0.43      0.21      0.28       158
           4       0.49      0.32      0.39       208
           5       0.74      0.41      0.53       469
           6       0.65      0.60      0.63        88
           7       0.53      0.30      0.38       209
           8       0.56      0.45      0.50       260
           9       0.30      0.11      0.17       219
          10       0.35      0.19      0.24       219
          11       0.77      0.66      0.71       145
          12       0.44      0.24      0.31       158
          13       0.66      0.33      0.44       266
          14       0.52      0.36      0.43       412
          15       0.50      0.38      0.43       162
          16       0.64      0.44      0.52       639
          17       0.46    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
#prediction et score pour notre val
y_pred_val = ovr_use.predict(X_val)
print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       0.58      0.50      0.54       121
           1       0.78      0.56      0.65       656
           2       0.45      0.37      0.40       132
           3       0.49      0.30      0.37       137
           4       0.44      0.24      0.31       185
           5       0.71      0.42      0.53       499
           6       0.50      0.50      0.50        64
           7       0.61      0.35      0.44       213
           8       0.59      0.45      0.51       284
           9       0.40      0.12      0.19       230
          10       0.37      0.23      0.28       213
          11       0.78      0.65      0.71       149
          12       0.44      0.20      0.27       171
          13       0.62      0.36      0.46       298
          14       0.57      0.35      0.43       457
          15       0.36      0.30      0.33       134
          16       0.67      0.43      0.52       621
          17       0.49    

c:\Users\Rémi\Documents\Openclassroom\IML\Projet 5\venvP52\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
